In [55]:
import requests
import os
import io
import json
from datetime import datetime
from confluent_kafka import Producer, Consumer, KafkaException
from minio import Minio
import uuid

api = "https://api.olhovivo.sptrans.com.br/v2.1"

def autenticarOlhoVivo():
    apikey = "[insira o token criado]" #INSERIR TOKEN CRIADO NO SITE OLHO VIVO
    endpoint = '/Login/Autenticar?token=' + apikey
    url = api + endpoint
    payload = {}
    headers = {}

    response = requests.request("POST", url, headers=headers, data=payload)
    get_cookie = response.cookies
    cookie = get_cookie.get("apiCredentials")
    print("Autenticando API Olho Vivo")
    return cookie


def linhasBuscar(cookie):
    endpoint = '/Linha/Buscar?termosBusca=8020' #Use case Linha: 8000, 8020, 8050
    url = api + endpoint
    payload = {}
    headers = {'Cookie': 'apiCredentials=' + cookie}
    response = requests.request("GET", url, headers=headers)
    print("Buscando linhas")
    #return response.json()
    
    topic = 'linhas_buscar'
    data = response.json()
    ingestaoKafka(data, topic)

    
def paradasBuscarParadasPorLinha(cookie):
    endpoint = '/Parada/BuscarParadasPorLinha?codigoLinha=1427' #Use case CodigoLinha: 35288, 1427
    url = api + endpoint
    payload = {}
    headers = {'Cookie': 'apiCredentials=' + cookie}
    response = requests.request("GET", url, headers=headers)
    print("Buscando paradas por linha")
    topic = 'paradas_busca_por_linha'
    data = response.json()
    ingestaoKafka(data, topic)


def posicaoVeiculosBuscar(cookie):
    endpoint = '/Posicao'
    url = api + endpoint
    payload = {}
    headers = {'Cookie': 'apiCredentials=' + cookie}
    response = requests.request("GET", url, headers=headers)
    print("Buscando posição dos veículos")
    topic = 'posicao_veiculo_busca'
    data = response.json()
    ingestaoKafka(data, topic)
    
    
def ingestaoKafka(data, topic):
    print("Ingestão kafka")
    broker = 'kafka-broker:9092'
    producer = Producer({
    'bootstrap.servers' : broker
    })
    producer.produce(topic, json.dumps(data))
    producer.flush()
    #print(f"Message sent to topic '{topic}': {json.dumps(data)}")
    print(f"Message sent to topic '{topic}' processed")
    
    
def workflow():
    cookie = autenticarOlhoVivo()
    linhasBuscar(cookie)
    #posicaoVeiculosBuscar(cookie)  ##KafkaException: KafkaError{code=MSG_SIZE_TOO_LARGE,val=10,str="Unable to produce message: Broker: Message size too large"}
    paradasBuscarParadasPorLinha(cookie)
    #data = linhasBuscar(cookie)
    #ingestaoKafka(data)
    
    #data = paradasBuscarParadasPorLinha(cookie)
    #ingestaoKafka(data)    

workflow()

Autenticando API Olho Vivo
Buscando linhas
Ingestão kafka
Message sent to topic 'linhas_buscar' processed
Buscando paradas por linha
Ingestão kafka
Message sent to topic 'paradas_busca_por_linha' processed
